Import Neccessary Libraries:

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt

Time To Maturity Regressions:

All 85 CBs:

In [ ]:
# Load the CB fixed parameters file
cb_fixed_file = 'CB fixed parameters-4.xlsx'  # Your CB fixed parameters file name
cb_df = pd.read_excel(cb_fixed_file)

# Load the Adjusted Volatility file and get the sheet names
adjusted_vol_file = 'Adjusted Volatility.xlsx'  # Your Adjusted Volatility file name
adjusted_vol_xls = pd.ExcelFile(adjusted_vol_file)
sheet_names = adjusted_vol_xls.sheet_names[:85]  # Get the first 85 sheet names

# Define a function to run regression for a subset of data
def run_regression(independent_var, dependent_var, include_constant=True):
    if include_constant:
        independent_var = sm.add_constant(independent_var)
    model = sm.OLS(dependent_var, independent_var).fit()
    return model

# Create an empty list to store data from all sheets
combined_data = []

# Loop through the first 85 sheets in Adjusted Volatility file
for sheet_name in sheet_names:
    print(f"\nProcessing sheet: {sheet_name}")
    
    # Check if the sheet name corresponds to a header in the CB fixed parameters-4 file
    column_headers = cb_df.columns.values
    
    if sheet_name in column_headers:
        # Locate the index of the sheet name in the column headers
        sheet_index = list(cb_df.columns).index(sheet_name)
        
        # Get the maturity date from row 6 (index 6)
        maturity_date = cb_df.iloc[6, sheet_index]

        # Load the data from the Adjusted Volatility file (current sheet)
        sheet_df = pd.read_excel(adjusted_vol_file, sheet_name=sheet_name)

        # Ensure that the date column exists and is in datetime format
        sheet_df['date'] = pd.to_datetime(sheet_df['date'], errors='coerce')

        # Subtract the maturity date from each data point in the 'date' column
        sheet_df['Time to Maturity'] = (maturity_date - sheet_df['date']).dt.days

        # Add the Time to Maturity Classification column
        def classify_time_to_maturity(days):
            if days < 365:
                return "<1 year"
            elif 365 <= days < 730:
                return "<2 years"
            elif 730 <= days < 1095:
                return "<3 years"
            elif 1095 <= days < 1460:
                return "<4 years"
            elif 1460 <= days < 1825:
                return "<5 years"
            elif 1825 <= days < 2190:
                return "<6 years"
            elif 2190 <= days < 2555:
                return "<7 years"
            elif 2555 <= days < 3650:
                return "<10 years"
            else:
                return ">10 years"

        # Apply the classification function to the 'Time to Maturity' column
        sheet_df['Time to Maturity Classification'] = sheet_df['Time to Maturity'].apply(classify_time_to_maturity)

        # Append the current sheet's data to the combined data list
        combined_data.append(sheet_df)
    else:
        print(f"Sheet {sheet_name} does not have a corresponding header in CB fixed parameters-4.")

# Combine all sheets' data into one DataFrame
combined_df = pd.concat(combined_data, ignore_index=True)

# Create an empty list to store the regression results
regression_results = []

# Run the regression for the different time to maturity classifications on the combined data
for classification in ['<1 year', '<2 years', '<3 years', '<4 years', '<5 years', '<6 years', '<7 years', '<10 years', '>10 years']:
    print(f"\nRunning regression for Time to Maturity classification: {classification}")
    
    # Filter the combined data for the specific Time to Maturity classification
    subset_df = combined_df[combined_df['Time to Maturity Classification'] == classification]
    
    if not subset_df.empty:
        # Log transformation of the dependent variable (BestIVOL)
        subset_df['Log BestIVOL'] = np.log(subset_df['BestIVOL'])
        subset_df['Log IVOL'] = np.log(subset_df['IVOL'])
        
        # Run regression with constant (log of BestIVOL as dependent variable)
        model_with_constant = run_regression(subset_df['Log IVOL'], subset_df['Log BestIVOL'], include_constant=True)
        
        # Run regression without constant (log of BestIVOL as dependent variable)
        model_without_constant = run_regression(subset_df['Log IVOL'], subset_df['Log BestIVOL'], include_constant=False)
        
        # Extract necessary information from the regression result (with constant)
        constant = model_with_constant.params.get('const', None)
        coeff_ivol_with_constant = model_with_constant.params.get('Log IVOL', None)
        adj_r2_with_constant = model_with_constant.rsquared_adj
        n_obs_with_constant = len(model_with_constant.model.endog)  # Number of observations
        
        # Extract necessary information from the regression result (without constant)
        coeff_ivol_without_constant = model_without_constant.params.get('Log IVOL', None)
        adj_r2_without_constant = model_without_constant.rsquared_adj
        n_obs_without_constant = len(model_without_constant.model.endog)  # Number of observations
        
        # Store the results in a list
        regression_results.append({
            'Time to Maturity Classification': classification,
            'Observations': n_obs_with_constant,
            'Constant Coefficient': constant,
            'IVOL Coefficient (with constant)': coeff_ivol_with_constant,
            'Adj. R^2 (with constant)': adj_r2_with_constant,
            'IVOL Coefficient (without constant)': coeff_ivol_without_constant,
            'Adj. R^2 (without constant)': adj_r2_without_constant
        })
    else:
        print(f"No data points for classification: {classification}")

# Create a DataFrame from the regression results
results_df = pd.DataFrame(regression_results)

# Display the resulting table
print("\nRegression Results Table:")
results_df

Scatter Plots:

In [ ]:
# Loop through each Time to Maturity classification to create the plots
for classification in ['<1 year', '<2 years', '<3 years', '<4 years', '<5 years', '<6 years', '<7 years', '<10 years', '>10 years']:
    # Filter the combined data for the specific Time to Maturity classification
    subset_df = combined_df[combined_df['Time to Maturity Classification'] == classification]
    
    if not subset_df.empty:
        # Independent variable: IVOL
        independent_var = subset_df['IVOL'].to_numpy()
        # Dependent variable: BestIVOL
        dependent_var = subset_df['BestIVOL'].to_numpy()
        
        # --- Standard Regressions ---
        
        # Without constant
        model_without_constant = sm.OLS(dependent_var, independent_var).fit()
        predicted_bestivol_without_constant = model_without_constant.predict(independent_var)
        
        # With constant
        independent_var_with_constant = sm.add_constant(independent_var)  # Add constant (intercept) term
        model_with_constant = sm.OLS(dependent_var, independent_var_with_constant).fit()
        predicted_bestivol_with_constant = model_with_constant.predict(independent_var_with_constant)
        
        # --- Log-Log Regressions ---
        
        # Log Transformation (only on positive values)
        positive_mask = (independent_var > 0) & (dependent_var > 0)
        log_independent_var = np.log(independent_var[positive_mask])
        log_dependent_var = np.log(dependent_var[positive_mask])

        # Log-log regression without constant
        model_log_no_const = sm.OLS(log_dependent_var, log_independent_var).fit()
        predicted_log_bestivol_no_const = model_log_no_const.predict(log_independent_var)
        
        # Log-log regression with constant
        log_independent_var_with_constant = sm.add_constant(log_independent_var)
        model_log_with_const = sm.OLS(log_dependent_var, log_independent_var_with_constant).fit()
        predicted_log_bestivol_with_const = model_log_with_const.predict(log_independent_var_with_constant)

        # Convert predictions back to normal scale
        predicted_bestivol_log_no_const = np.exp(predicted_log_bestivol_no_const)
        predicted_bestivol_log_with_const = np.exp(predicted_log_bestivol_with_const)
        
        # --- 1. Scatter Plot for Standard Regressions ---
        plt.figure(figsize=(10, 6))
        
        # Scatter plot for the actual data points
        plt.scatter(subset_df['IVOL'], subset_df['BestIVOL'], alpha=0.6, color='red', label='BestIVOL vs IVOL')
        
        # Scatter plot for the regression without constant
        plt.scatter(subset_df['IVOL'], predicted_bestivol_without_constant, alpha=0.6, color='blue', label='Estimated BestIVOL (without constant)')
        
        # Scatter plot for the regression with constant
        plt.scatter(subset_df['IVOL'], predicted_bestivol_with_constant, alpha=0.6, color='green', label='Estimated BestIVOL (with constant)')
        
        # Titles and labels
        plt.title(f'Standard Regressions: IVOL vs BestIVOL for {classification}')
        plt.xlabel('Implied Volatility (IVOL)')
        plt.ylabel('Best Implied Volatility (BestIVOL)')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # --- 2. Scatter Plot for Log-Log Regressions ---
        plt.figure(figsize=(10, 6))
        
        # Scatter plot for the actual data points (positive values only)
        plt.scatter(independent_var[positive_mask], dependent_var[positive_mask], alpha=0.6, color='red', label='Actual BestIVOL')
        
        # Scatter plot for the log-log regression without constant
        plt.scatter(independent_var[positive_mask], predicted_bestivol_log_no_const, alpha=0.6, color='purple', label='Log-Log Estimated BestIVOL (without constant)')
        
        # Scatter plot for the log-log regression with constant
        plt.scatter(independent_var[positive_mask], predicted_bestivol_log_with_const, alpha=0.6, color='orange', label='Log-Log Estimated BestIVOL (with constant)')
        
        # Titles and labels
        plt.title(f'Log-Log Regressions: IVOL vs BestIVOL for {classification}')
        plt.xlabel('Implied Volatility (IVOL)')
        plt.ylabel('Best Implied Volatility (BestIVOL)')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # --- 3. Residual Plot for Standard Regressions ---
        plt.figure(figsize=(10, 6))
        
        # Residuals for standard regression
        residuals_without_constant = dependent_var - predicted_bestivol_without_constant
        residuals_with_constant = dependent_var - predicted_bestivol_with_constant
        
        plt.scatter(subset_df['IVOL'], residuals_without_constant, alpha=0.6, color='purple', label='Residuals (without constant)')
        plt.scatter(subset_df['IVOL'], residuals_with_constant, alpha=0.6, color='orange', label='Residuals (with constant)')
        
        # Add horizontal line at y=0 for reference
        plt.axhline(y=0, color='black', linestyle='--')
        
        # Titles and labels
        plt.title(f'Residuals (Standard Regressions) for {classification}')
        plt.xlabel('Implied Volatility (IVOL)')
        plt.ylabel('Residuals')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # --- 4. Residual Plot for Log-Log Regressions (Original Units) ---
        plt.figure(figsize=(10, 6))

        # Residuals for log-log regression in the original units (convert from log scale)
        residuals_log_no_const_original = np.exp(log_dependent_var) - np.exp(predicted_log_bestivol_no_const)
        residuals_log_with_const_original = np.exp(log_dependent_var) - np.exp(predicted_log_bestivol_with_const)

        plt.scatter(independent_var[positive_mask], residuals_log_no_const_original, alpha=0.6, color='brown', label='Log-Log Residuals (without constant)')
        plt.scatter(independent_var[positive_mask], residuals_log_with_const_original, alpha=0.6, color='cyan', label='Log-Log Residuals (with constant)')

        # Add horizontal line at y=0 for reference
        plt.axhline(y=0, color='black', linestyle='--')

        # Titles and labels
        plt.title(f'Residuals (Log-Log Regressions) for {classification} (Original Units)')
        plt.xlabel('Implied Volatility (IVOL)')
        plt.ylabel('Residuals (Original Units)')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # Print regression summaries
        print(f'Log-Log Regression Summary (without constant) for {classification}:\n', model_log_no_const.summary())
        print(f'Log-Log Regression Summary (with constant) for {classification}:\n', model_log_with_const.summary())
        
    else:
        print(f"No data points for scatter plot: {classification}")

Industry: REIT Mortgage

In [ ]:
# Load the CB fixed parameters file
cb_fixed_file = 'CB fixed parameters-4.xlsx'  # Your CB fixed parameters file name
cb_df = pd.read_excel(cb_fixed_file)

# Load the Adjusted Volatility file and get the sheet names
adjusted_vol_file = 'Adjusted Volatility.xlsx'  # Your Adjusted Volatility file name
adjusted_vol_xls = pd.ExcelFile(adjusted_vol_file)
sheet_names = adjusted_vol_xls.sheet_names[:85]  # Get the first 85 sheet names

# Define a function to run regression for a subset of data
def run_regression(independent_var, dependent_var, include_constant=True):
    if include_constant:
        independent_var = sm.add_constant(independent_var)
    model = sm.OLS(dependent_var, independent_var).fit()
    return model

# Create an empty list to store data from all sheets
combined_data = []

# Loop through the first 85 sheets in Adjusted Volatility file
for sheet_name in sheet_names:
    print(f"\nProcessing sheet: {sheet_name}")
    
    # Check if the sheet name corresponds to a header in the CB fixed parameters-4 file
    column_headers = cb_df.columns.values
    
    if sheet_name in column_headers:
        # Locate the index of the sheet name in the column headers
        sheet_index = list(cb_df.columns).index(sheet_name)
        
        # Get the industry classification from index 10
        industry_classification = cb_df.iloc[8, sheet_index]
        
        # Only process sheets in the "REIT - Mortgage" industry
        if industry_classification == "REIT - Mortgage":
            # Get the maturity date from row 6 (index 6)
            maturity_date = cb_df.iloc[6, sheet_index]
            
            # Load the data from the Adjusted Volatility file (current sheet)
            sheet_df = pd.read_excel(adjusted_vol_file, sheet_name=sheet_name)
            
            # Ensure that the date column exists and is in datetime format
            sheet_df['date'] = pd.to_datetime(sheet_df['date'], errors='coerce')
            
            # Subtract the maturity date from each data point in the 'date' column
            sheet_df['Time to Maturity'] = (maturity_date - sheet_df['date']).dt.days
            
            # Add the Time to Maturity Classification column
            def classify_time_to_maturity(days):
                if days < 365:
                    return "<1 year"
                elif 365 <= days < 730:
                    return "<2 years"
                elif 730 <= days < 1095:
                    return "<3 years"
                elif 1095 <= days < 1460:
                    return "<4 years"
                elif 1460 <= days < 1825:
                    return "<5 years"
                elif 1825 <= days < 2190:
                    return "<6 years"
                elif 2190 <= days < 2555:
                    return "<7 years"
                elif 2555 <= days < 3650:
                    return "<10 years"
                else:
                    return ">10 years"
            
            # Apply the classification function to the 'Time to Maturity' column
            sheet_df['Time to Maturity Classification'] = sheet_df['Time to Maturity'].apply(classify_time_to_maturity)
            
            # Append the current sheet's data to the combined data list
            combined_data.append(sheet_df)
        else:
            print(f"Skipping sheet {sheet_name} as it does not belong to 'REIT - Mortgage' industry.")
    else:
        print(f"Sheet {sheet_name} does not have a corresponding header in CB fixed parameters-4.")

# Combine all sheets' data into one DataFrame
combined_df = pd.concat(combined_data, ignore_index=True)

# Create an empty list to store the regression results
regression_results = []

# Run the regression for the different time to maturity classifications on the combined data
for classification in ['<1 year', '<2 years', '<3 years', '<4 years','<5 years', '<6 years','<7 years','<10 years','>10 years']:
    print(f"\nRunning regression for Time to Maturity classification: {classification}")
    
    # Filter the combined data for the specific Time to Maturity classification
    subset_df = combined_df[combined_df['Time to Maturity Classification'] == classification]
    
    if not subset_df.empty:
        # Run regression with constant
        model_with_constant = run_regression(subset_df['IVOL'], subset_df['BestIVOL'], include_constant=True)
        
        # Run regression without constant
        model_without_constant = run_regression(subset_df['IVOL'], subset_df['BestIVOL'], include_constant=False)
        
        # Extract necessary information from the regression result (with constant)
        constant = model_with_constant.params.get('const', None)
        coeff_ivol_with_constant = model_with_constant.params.get('IVOL', None)
        adj_r2_with_constant = model_with_constant.rsquared_adj
        n_obs_with_constant = len(model_with_constant.model.endog)  # Number of observations
        
        # Extract necessary information from the regression result (without constant)
        coeff_ivol_without_constant = model_without_constant.params.get('IVOL', None)
        adj_r2_without_constant = model_without_constant.rsquared_adj
        n_obs_without_constant = len(model_without_constant.model.endog)  # Number of observations
        
        # Store the results in a list
        regression_results.append({
            'Time to Maturity Classification': classification,
            'Observations': n_obs_with_constant,
            'Constant Coefficient': constant,
            'IVOL Coefficient (with constant)': coeff_ivol_with_constant,
            'Adj. R^2 (with constant)': adj_r2_with_constant,
            'IVOL Coefficient (without constant)': coeff_ivol_without_constant,
            'Adj. R^2 (without constant)': adj_r2_without_constant
        })
    else:
        print(f"No data points for classification: {classification}")

# Create a DataFrame from the regression results
results_df = pd.DataFrame(regression_results)

# Display the resulting table
print("\nRegression Results Table:")
results_df

Moneyness Regressions:

All 85 CBs:

In [ ]:
# Load the CB fixed parameters file
cb_fixed_file = 'CB fixed parameters-4.xlsx'  # Your CB fixed parameters file name
cb_df = pd.read_excel(cb_fixed_file)

# Load the Adjusted Volatility file and get the sheet names
adjusted_vol_file = 'Adjusted Volatility.xlsx'  # Your Adjusted Volatility file name
adjusted_vol_xls = pd.ExcelFile(adjusted_vol_file)
sheet_names = adjusted_vol_xls.sheet_names[:85]  # Get the first 85 sheet names

# Define a function to classify moneyness
def classify_moneyness(stock_price, conversion_price):
    if stock_price > 1.5 * conversion_price:
        return 'Far ITM'  # In the Money
    elif 1.1 * conversion_price <= stock_price <= 1.5 * conversion_price:
        return 'Close ITM'
    elif 0.9 * conversion_price <= stock_price <= 1.1 * conversion_price:
        return 'ATM'  # At the Money
    elif 0.5 * conversion_price <= stock_price <= 0.9 * conversion_price:
        return "Close OTM"
    else:
        return 'Far OTM'  # Out of the Money

# Create an empty list to store data from all sheets
combined_data = []

# Loop through all sheets in the Adjusted Volatility file
for sheet_name in sheet_names:
    print(f"\nProcessing sheet: {sheet_name}")
    
    # Check if the sheet name corresponds to a header in the CB fixed parameters-4 file
    column_headers = cb_df.columns.values
    
    if sheet_name in column_headers:
        # Locate the index of the sheet name in the column headers
        sheet_index = list(cb_df.columns).index(sheet_name)
        
        # Get the conversion price from row 2 (index 2) in CB fixed parameters
        conversion_price = cb_df.iloc[2, sheet_index]
        
        # Load the data from the Adjusted Volatility file (current sheet)
        sheet_df = pd.read_excel(adjusted_vol_file, sheet_name=sheet_name)
        
        # Ensure that the date column exists and is in datetime format
        sheet_df['date'] = pd.to_datetime(sheet_df['date'], errors='coerce')
        
        # Calculate moneyness and classify based on the stock price and conversion price
        sheet_df['Moneyness'] = sheet_df['S'].apply(lambda x: classify_moneyness(x, conversion_price))
        
        # Add the sheet's data to the combined list
        combined_data.append(sheet_df)
    else:
        print(f"Skipping sheet {sheet_name} as it does not have a corresponding header in CB fixed parameters-4.")

# Combine all sheets' data into one DataFrame
combined_df = pd.concat(combined_data, ignore_index=True)

# Define a function to perform the regression and return results
def run_log_regression(df, with_constant=True):
    # Apply log transformation to both IVOL and BestIVOL
    df['log_IVOL'] = np.log(df['IVOL'])
    df['log_BestIVOL'] = np.log(df['BestIVOL'])
    
    X = df['log_IVOL']
    y = df['log_BestIVOL']
    
    if with_constant:
        X = sm.add_constant(X)  # Add constant for regression with intercept
        model = sm.OLS(y, X).fit()
        return model.params[0], model.params[1], model.rsquared_adj  # Constant, IVOL Coefficient, and Adjusted R-squared
    else:
        model = sm.OLS(y, X).fit()
        return model.params[0], model.rsquared_adj  # Coefficients and Adjusted R-squared without constant

# Create an empty list to store the results
regression_results = []

# Loop through the moneyness categories
for moneyness in ['Far ITM', 'Close ITM', 'ATM', 'Close OTM', 'Far OTM']:
    print(f"\nProcessing moneyness category: {moneyness}")
    
    # Filter the data for the current moneyness category
    moneyness_df = combined_df[combined_df['Moneyness'] == moneyness]
    
    if not moneyness_df.empty:
        # Get the number of observations
        num_observations = len(moneyness_df)
        
        # Regression with constant
        constant_with_const, ivol_coeff_with_const, adj_r2_with_const = run_log_regression(moneyness_df, with_constant=True)
        
        # Regression without constant
        ivol_coeff_without_const, adj_r2_without_const = run_log_regression(moneyness_df, with_constant=False)
        
        # Store results including number of observations and the constant coefficient
        regression_results.append({
            'Moneyness': moneyness,
            'Observations': num_observations,
            'Constant Coefficient (with constant)': constant_with_const,
            'IVOL Coefficient (with constant)': ivol_coeff_with_const,
            'Adj. R^2 (with constant)': adj_r2_with_const,
            'IVOL Coefficient (without constant)': ivol_coeff_without_const,
            'Adj. R^2 (without constant)': adj_r2_without_const
        })

# Create a DataFrame to display the results
results_df = pd.DataFrame(regression_results)

# Display the resulting regression results
print("\nRegression Results Table:")
print(results_df)

Scatter Plots:

In [ ]:
# Loop through the moneyness categories
for moneyness in ['Far ITM', 'Close ITM', 'ATM', 'Close OTM', 'Far OTM']:
    # Filter the data for the current moneyness category
    moneyness_df = combined_df[combined_df['Moneyness'] == moneyness]
    
    if not moneyness_df.empty:
        # Prepare data for regression
        X = moneyness_df['IVOL'].to_numpy()
        y = moneyness_df['BestIVOL'].to_numpy()
        
        # Regression with constant
        X_const = sm.add_constant(X)
        model_with_const = sm.OLS(y, X_const).fit()
        intercept, slope = model_with_const.params
        
        # Estimate BestIVOL using the regression with constant
        estimated_with_const = intercept + slope * X
        
        # Regression without constant
        model_without_const = sm.OLS(y, X).fit()
        slope_no_const = model_without_const.params[0]
        
        # Estimate BestIVOL using the regression without constant
        estimated_without_const = slope_no_const * X
        
        # Calculate residuals for both models
        residuals_with_const = y - estimated_with_const
        residuals_without_const = y - estimated_without_const
        
        # Plot all scatter plots
        plt.figure(figsize=(10, 12))
        
        # 1st subplot: Actual BestIVOL vs IVOL, IVOL vs IVOL, and Estimated BestIVOL
        plt.subplot(2, 1, 1)
        plt.scatter(moneyness_df['IVOL'], moneyness_df['BestIVOL'], color='blue', label='Actual BestIVOL', alpha=0.7)
        plt.scatter(moneyness_df['IVOL'], moneyness_df['IVOL'], color='green', label='IVOL vs IVOL', alpha=0.5)
        plt.scatter(moneyness_df['IVOL'], estimated_with_const, color='red', label='Estimated (with constant)', alpha=0.7)
        plt.scatter(moneyness_df['IVOL'], estimated_without_const, color='orange', label='Estimated (without constant)', alpha=0.7)
        plt.title(f'IVOL and BestIVOL Estimates - {moneyness}')
        plt.xlabel('IVOL')
        plt.ylabel('Volatility')
        plt.legend()
        plt.grid(True)
        
        # 2nd subplot: Residuals plot
        plt.subplot(2, 1, 2)
        plt.scatter(moneyness_df['IVOL'], residuals_with_const, color='red', label='Residuals (with constant)', alpha=0.7)
        plt.scatter(moneyness_df['IVOL'], residuals_without_const, color='orange', label='Residuals (without constant)', alpha=0.7)
        plt.axhline(0, color='black', linestyle='--')  # Line at 0 for reference
        plt.title(f'Residuals for {moneyness}')
        plt.xlabel('IVOL')
        plt.ylabel('Residuals')
        plt.legend()
        plt.grid(True)
        
        # Show the plots
        plt.tight_layout()
        plt.show()

Industry: REIT Mortgage

In [ ]:
# Load the CB fixed parameters file
cb_fixed_file = 'CB fixed parameters-4.xlsx'  # Your CB fixed parameters file name
cb_df = pd.read_excel(cb_fixed_file)

# Load the Adjusted Volatility file and get the sheet names
adjusted_vol_file = 'Adjusted Volatility.xlsx'  # Your Adjusted Volatility file name
adjusted_vol_xls = pd.ExcelFile(adjusted_vol_file)
sheet_names = adjusted_vol_xls.sheet_names[:85]  # Get the first 85 sheet names

# Define a function to classify moneyness
def classify_moneyness(stock_price, conversion_price):
    if stock_price > 1.5 * conversion_price:
        return 'Far ITM'  # In the Money
    elif 1.1 * conversion_price <= stock_price <= 1.5 * conversion_price:
        return 'Close ITM'
    elif 0.9 * conversion_price <= stock_price <= 1.1 * conversion_price:
        return 'ATM'  # At the Money
    elif 0.5 * conversion_price <= stock_price <= 0.9 * conversion_price:
        return "Close OTM"
    else:
        return 'Far OTM'  # Out of the Money

# Create an empty list to store data from all sheets
combined_data = []

# Loop through all sheets in the Adjusted Volatility file
for sheet_name in sheet_names:
    print(f"\nProcessing sheet: {sheet_name}")
    
    # Check if the sheet name corresponds to a header in the CB fixed parameters-4 file
    column_headers = cb_df.columns.values
    
    if sheet_name in column_headers:
        # Locate the index of the sheet name in the column headers
        sheet_index = list(cb_df.columns).index(sheet_name)
        
        # Get the industry classification from index 10
        industry_classification = cb_df.iloc[8, sheet_index]
        
        # Only process sheets in the "REIT - Mortgage" industry
        if industry_classification == "REIT - Mortgage":
            # Get the conversion price from row 2 (index 2) in CB fixed parameters
            conversion_price = cb_df.iloc[2, sheet_index]
            
            # Load the data from the Adjusted Volatility file (current sheet)
            sheet_df = pd.read_excel(adjusted_vol_file, sheet_name=sheet_name)
            
            # Ensure that the date column exists and is in datetime format
            sheet_df['date'] = pd.to_datetime(sheet_df['date'], errors='coerce')
            
            # Calculate moneyness and classify based on the stock price and conversion price
            sheet_df['Moneyness'] = sheet_df['S'].apply(lambda x: classify_moneyness(x, conversion_price))
            
            # Append the current sheet's data to the combined data list
            combined_data.append(sheet_df)
        else:
            print(f"Skipping sheet {sheet_name} as it does not belong to 'REIT - Mortgage' industry.")
    else:
        print(f"Sheet {sheet_name} does not have a corresponding header in CB fixed parameters-4.")

# Combine all sheets' data into one DataFrame
combined_df = pd.concat(combined_data, ignore_index=True)

# Define a function to perform the regression and return results
def run_regression(df, with_constant=True):
    X = df['IVOL']
    y = df['BestIVOL']
    
    if with_constant:
        X = sm.add_constant(X)  # Add constant for regression with intercept
        model = sm.OLS(y, X).fit()
        return model.params[0], model.params[1], model.rsquared_adj  # Constant, IVOL Coefficient, and Adjusted R-squared
    else:
        model = sm.OLS(y, X).fit()
        return model.params[0], model.rsquared_adj  # Coefficients and Adjusted R-squared without constant

# Create an empty list to store the results
regression_results = []

# Loop through the moneyness categories
for moneyness in ['Far ITM', 'Close ITM', 'ATM', 'Close OTM', 'Far OTM']:
    print(f"\nProcessing moneyness category: {moneyness}")
    
    # Filter the data for the REIT - Mortgage industry and current moneyness category
    moneyness_df = combined_df[(combined_df['Moneyness'] == moneyness)]
    
    if not moneyness_df.empty:
        # Get the number of observations
        num_observations = len(moneyness_df)
        
        # Regression with constant
        constant_with_const, ivol_coeff_with_const, adj_r2_with_const = run_regression(moneyness_df, with_constant=True)
        
        # Regression without constant
        ivol_coeff_without_const, adj_r2_without_const = run_regression(moneyness_df, with_constant=False)
        
        # Store results including number of observations and the constant coefficient
        regression_results.append({
            'Moneyness': moneyness,
            'Observations': num_observations,
            'Constant Coefficient (with constant)': constant_with_const,
            'IVOL Coefficient (with constant)': ivol_coeff_with_const,
            'Adj. R^2 (with constant)': adj_r2_with_const,
            'IVOL Coefficient (without constant)': ivol_coeff_without_const,
            'Adj. R^2 (without constant)': adj_r2_without_const
        })

# Create a DataFrame to display the results
results_df = pd.DataFrame(regression_results)

# Display the results
print("\nRegression Results for REIT - Mortgage Industry (separated by Moneyness):")
results_df